In [22]:
import requests
from urllib.parse import urlencode
from pymongo import MongoClient
import time


class Toutiao(object):
    def __init__(self, name):
        self.name = name  # 获取要爬取的搜索内容相应的关键字
        self.base_url = "https://www.toutiao.com/api/search/content/?"
        self.headers = {
            "cookie":
            "tt_webid=6736143139453601287; WEATHER_CITY=%E5%8C%97%E4%BA%AC; tt_webid=6736143139453601287; csrftoken=1173d1e59061616313ba56775c3f5118; _ga=GA1.2.73970871.1568380906; __tasessionId=p6fwyaarb1569309235846; s_v_web_id=9d78d95a5883b8e8a9363aa0386c9e3a",
            "referer":
            "https://www.toutiao.com/search/?keyword=%E8%A1%97%E6%8B%8D",
            "user-agent":
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
            "x-requested-with": "XMLHttpRequest"
        }
        # 初始化mongoDB客户端，并指定collection为 toutiao
        client = MongoClient(host='localhost', port=27017)
        db = client['TouTiao']
        self.collection = db['toutiao']

    def parse_url(self, url):
        """发送请求，获取响应"""
        print(url)
        try:
            response = requests.get(url, headers=self.headers)
            if response.status_code == 200:
                return response.json()
        except requests.ConnectionError as e:
            print("Error is ", e.args)

    def get_data(self, html_json):
        """根据传进来的json字典，提取相应的数据"""
        if html_json['count']:  # 判断新闻的数量
            items = html_json['data']
            for item in items:
                img_dict = {}
                if item.get('has_image', None):  # 判断数据中是否含有图片
                    img_dict['title'] = item.get('title', None)  # 获取图片的title
                    img_dict['image_list'] = item.get(
                        'image_list')  # 获取所有图片的url，返回一个字典
                    yield img_dict

    def run(self):
        """实现主程序"""
        has_more = 1  # 先假设网页有多个ajax请求
        page = 0  # 初始化网页的页码
        while has_more:
            # 构造url
            self.params = {
                "aid": "24",
                "app_name": "web_search",
                "offset": page * 20,
                "format": "json",
                "keyword": self.name,
                "autoload": "true",
                "count": "20",
                "en_qc": "1",
                "cur_tab": "1",
                "from": "search_tab",
                "pd": "synthesis",
                "timestamp": int(time.time() * 1000)  # 根据ajax请求，模拟相应的时间戳
            }
            page += 1
            url = self.base_url + urlencode(self.params)
            # 发送请求获取响应
            html_json = self.parse_url(url)
            if html_json['count']:
                has_more = html_json['has_more']
            # 提取数据
            img_dict_generator = self.get_data(html_json)
            # 保存数据
            for img in img_dict_generator:
                try:
                    self.collection.insert_one(img)
                    print('保存成功')
                except:
                    print("保存过了")
            time.sleep(2)

In [23]:
if __name__ == "__main__":
    toutiao_jiepai = Toutiao("街拍")
    html_json = toutiao_jiepai.run()

https://www.toutiao.com/api/search/content/?aid=24&app_name=web_search&offset=0&format=json&keyword=%E8%A1%97%E6%8B%8D&autoload=true&count=20&en_qc=1&cur_tab=1&from=search_tab&pd=synthesis&timestamp=1569309399036
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
https://www.toutiao.com/api/search/content/?aid=24&app_name=web_search&offset=20&format=json&keyword=%E8%A1%97%E6%8B%8D&autoload=true&count=20&en_qc=1&cur_tab=1&from=search_tab&pd=synthesis&timestamp=1569309423307
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
https://www.toutiao.com/api/search/content/?aid=24&app_name=web_search&offset=40&format=json&keyword=%E8%A1%97%E6%8B%8D&autoload=true&count=20&en_qc=1&cur_tab=1&from=search_tab&pd=synthesis&timestamp=1569309447697
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
保存成功
https://www.toutiao.com/api/search/content/?aid=24&app_name=web_search&offset=60&format=json&keywor

In [25]:
import re
from queue import Queue
from pymongo import MongoClient
import urllib
import os
import time


def decorator(func):
    def wrap(*args, **kwargs):
        start = time.time()
        ret = func(*args, **kwargs)
        end = time.time()
        print("spend time : {:.8f}".format(end-start))
        return ret
    return wrap

class ProduceUrl(threading.Thread):
    def __init__(self, filepath_q, url_q, collection, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filepath_q = filepath_q
        self.url_q = url_q
        self.collection = collection

    @decorator      
    def run(self):
        result = self.collection.aggregate([{"$project":{"_id":0}}])  # 获取图片数据
        for img in result:
            title = img.get('title')
            file_name = re.sub(r'[:：，。,\.?？！!\|"”“]', "", title)  # 去掉不规范字符，防止在写入文件时因为文件名而出错
            image_list = img.get('image_list')
            for index, image in enumerate(image_list, start=1):
                src = image.get('url')
                self.url_q.put(src)
                file_path = "../crawl_file/images/toutiao/" + file_name + str(index)+".jpg"
                self.filepath_q.put(file_path)
                

class ConsumerUrl(threading.Thread):
    def __init__(self, filepath_q, url_q, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filepath_q = filepath_q
        self.url_q = url_q
    
    @decorator
    def run(self):
        while True:
            src = self.url_q.get()
            file_path = self.filepath_q.get()
            if not os.path.exists(file_path):  # 根据文件名判断是否文件已经下载过了
                urllib.request.urlretrieve(src, file_path)
                print('保存成功' + file_path)
            if self.url_q.empty():
                break
        

                    
                    
def main():
    # 连接数据库，并指定collection
    client = MongoClient(host='localhost', port=27017)
    db = client['TouTiao']
    collection = db['toutiao']
    # 创建文件路径队列和url队列
    filepath_q = Queue(500)
    url_q = Queue(1000)
    # 创建生产者进程，生产文件路径和url
    threads = []
    for i in range(2):
        t_producer = ProduceUrl(filepath_q, url_q, collection)
        t_producer.start()
        threads.append(t_producer)
    for i in range(10):
        t_consumer = ConsumerUrl(filepath_q, url_q)
        t_consumer.start()
        threads.append(t_consumer)
    
    for thread in threads:
        thread.join()
    
if __name__ == "__main__":
    main()
    

保存成功../crawl_file/images/toutiao/街拍 性感美女身穿白色蕾丝内衣就敢出门路人眼都直了2.jpg
保存成功../crawl_file/images/toutiao/街拍 性感美女身穿白色蕾丝内衣就敢出门路人眼都直了3.jpg
保存成功../crawl_file/images/toutiao/北京街拍三里屯潮拍凸显与众不同的时尚街拍与众不同最时尚1.jpg
保存成功../crawl_file/images/toutiao/街拍 性感美女身穿白色蕾丝内衣就敢出门路人眼都直了4.jpg保存成功../crawl_file/images/toutiao/街拍一对跑步闺蜜走红汗水还挂在额头活力十足1.jpg保存成功../crawl_file/images/toutiao/北京街拍三里屯潮拍凸显与众不同的时尚街拍与众不同最时尚4.jpg

保存成功../crawl_file/images/toutiao/北京街拍三里屯潮拍凸显与众不同的时尚街拍与众不同最时尚3.jpg保存成功../crawl_file/images/toutiao/北京街拍三里屯潮拍凸显与众不同的时尚街拍与众不同最时尚2.jpg


保存成功../crawl_file/images/toutiao/街拍一对跑步闺蜜走红汗水还挂在额头活力十足2.jpg
保存成功../crawl_file/images/toutiao/街拍一对跑步闺蜜走红汗水还挂在额头活力十足3.jpg
保存成功../crawl_file/images/toutiao/街拍 性感美女身穿白色蕾丝内衣就敢出门路人眼都直了1.jpg保存成功../crawl_file/images/toutiao/街拍一对跑步闺蜜走红汗水还挂在额头活力十足5.jpg

保存成功../crawl_file/images/toutiao/街拍一对跑步闺蜜走红汗水还挂在额头活力十足4.jpg
保存成功../crawl_file/images/toutiao/街拍路人街头的美是最真实的尤其是夏天3.jpg
保存成功../crawl_file/images/toutiao/三里屯街拍偶遇长腿大胸网红于芷晴1.jpg
保存成功../crawl_file/images/toutiao/街拍路人街头的美是最真实的尤其是夏天1.jpg
保存成功../craw

保存成功../crawl_file/images/toutiao/成都街拍阔腿裤这样穿最时髦1.jpg
保存成功../crawl_file/images/toutiao/美女街拍目测只有90斤的清瘦妹子带着口罩也挡不住的魅力1.jpg
保存成功../crawl_file/images/toutiao/摄影街拍丨风姿绰约颜值和大长腿永远是杨幂的亮点1.jpg
保存成功../crawl_file/images/toutiao/伊万卡携老公参加婚礼穿斗篷裙180cm身材了不得抢足新娘风头3.jpg
保存成功../crawl_file/images/toutiao/成都街拍阔腿裤这样穿最时髦3.jpg
保存成功../crawl_file/images/toutiao/摄影街拍丨风姿绰约颜值和大长腿永远是杨幂的亮点2.jpg
保存成功../crawl_file/images/toutiao/三里屯街拍一位美丽的女孩有点像钟欣桐谁更美1.jpg
保存成功../crawl_file/images/toutiao/冬季最新韩国型人街拍合辑N张街拍带你学穿首尔Style1.jpg
保存成功../crawl_file/images/toutiao/冬季最新韩国型人街拍合辑N张街拍带你学穿首尔Style4.jpg
保存成功../crawl_file/images/toutiao/摄影街拍丨风姿绰约颜值和大长腿永远是杨幂的亮点4.jpg
保存成功../crawl_file/images/toutiao/摄影街拍丨风姿绰约颜值和大长腿永远是杨幂的亮点3.jpg
保存成功../crawl_file/images/toutiao/街拍三里屯的潮人就是时尚中的引领者3.jpg保存成功../crawl_file/images/toutiao/半个月大的婴儿拉粑粑表情好丰富好萌1.jpg保存成功../crawl_file/images/toutiao/冬季最新韩国型人街拍合辑N张街拍带你学穿首尔Style2.jpg


保存成功../crawl_file/images/toutiao/街拍三里屯的潮人就是时尚中的引领者2.jpg
保存成功../crawl_file/images/toutiao/冬季最新韩国型人街拍合辑N张街拍带你学穿首尔Style3.jpg
保存成功../crawl_file/ima

保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧4.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧6.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧9.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧11.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧10.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧8.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧13.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧5.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧12.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧7.jpg
保存成功../crawl_file/images/toutiao/给有追求的街拍摄影师的10个技巧15.jpg
保存成功../crawl_file/images/toutiao/成都街拍夏天穿衣选好适合自己的风格会更美4.jpg
保存成功../crawl_file/images/toutiao/成都街拍夏天穿衣选好适合自己的风格会更美3.jpg
保存成功../crawl_file/images/toutiao/成都街拍夏天穿衣选好适合自己的风格会更美2.jpg
保存成功../crawl_file/images/toutiao/成都街拍美女如云的城市彰显不一样的时尚美2.jpg
保存成功../crawl_file/images/toutiao/成都街拍美女如云的城市彰显不一样的时尚美1.jpg保存成功../crawl_file/images/toutiao/成都街拍夏天穿衣选好适合自己的风格会更美1.jpg

保存成功../crawl_file/images/toutiao/成都街


保存成功../crawl_file/images/toutiao/街拍夏季成群结队的美女你最爱哪个4.jpg
保存成功../crawl_file/images/toutiao/街拍夏季成群结队的美女你最爱哪个2.jpg保存成功../crawl_file/images/toutiao/重庆街拍睡衣风的小吊带连衣裙好看又任性4.jpg保存成功../crawl_file/images/toutiao/街拍路人三伏天不知道该如何选择穿搭不妨看看这几组图片1.jpg


保存成功../crawl_file/images/toutiao/街拍路人三伏天不知道该如何选择穿搭不妨看看这几组图片2.jpg
保存成功../crawl_file/images/toutiao/街拍路人三伏天不知道该如何选择穿搭不妨看看这几组图片3.jpg
保存成功../crawl_file/images/toutiao/街拍路人三伏天不知道该如何选择穿搭不妨看看这几组图片4.jpg
保存成功../crawl_file/images/toutiao/街拍路人青春时尚的穿搭参考让你越看越年轻美丽1.jpg
保存成功../crawl_file/images/toutiao/成都街拍穿出高颜值美感搭配参考出门逛街约会永远都适合1.jpg
保存成功../crawl_file/images/toutiao/街拍路人青春时尚的穿搭参考让你越看越年轻美丽2.jpg
保存成功../crawl_file/images/toutiao/成都街拍穿出高颜值美感搭配参考出门逛街约会永远都适合3.jpg
保存成功../crawl_file/images/toutiao/街拍路人青春时尚的穿搭参考让你越看越年轻美丽4.jpg
保存成功../crawl_file/images/toutiao/街拍不仅穿着舒适更显高还恰到妙处呈现女人魅力1.jpg
保存成功../crawl_file/images/toutiao/成都街拍穿出高颜值美感搭配参考出门逛街约会永远都适合4.jpg保存成功../crawl_file/images/toutiao/成都街拍穿出高颜值美感搭配参考出门逛街约会永远都适合2.jpg

保存成功../crawl_file/images/toutiao/街拍不仅穿着舒适更显高还恰到妙处呈现女人魅力3.jpg
保存成功..

保存成功../crawl_file/images/toutiao/「成都街拍」太古里美女冬季街拍抢眼撩男功力不只是嘴上说说2.jpg
保存成功../crawl_file/images/toutiao/批发市场街拍超短连衣裙美女印花裙也很美1.jpg保存成功../crawl_file/images/toutiao/摄影师不会告诉你的23种街拍小妙招1.jpg

保存成功../crawl_file/images/toutiao/「成都街拍」太古里美女冬季街拍抢眼撩男功力不只是嘴上说说1.jpg
保存成功../crawl_file/images/toutiao/批发市场街拍超短连衣裙美女印花裙也很美2.jpg
保存成功../crawl_file/images/toutiao/「成都街拍」太古里美女冬季街拍抢眼撩男功力不只是嘴上说说4.jpg
保存成功../crawl_file/images/toutiao/「成都街拍」太古里美女冬季街拍抢眼撩男功力不只是嘴上说说3.jpg
保存成功../crawl_file/images/toutiao/批发市场街拍超短连衣裙美女印花裙也很美4.jpg
保存成功../crawl_file/images/toutiao/杭州1.jpg
保存成功../crawl_file/images/toutiao/批发市场街拍超短连衣裙美女印花裙也很美3.jpg
保存成功../crawl_file/images/toutiao/长相普通就不能追到女神小伙经历告诉你缘分就是妙不可言1.jpg
保存成功../crawl_file/images/toutiao/街拍小姐姐的腿是有多细这么紧的牛仔裤都穿得下真腿精2.jpg
保存成功../crawl_file/images/toutiao/街拍小姐姐的腿是有多细这么紧的牛仔裤都穿得下真腿精3.jpg
保存成功../crawl_file/images/toutiao/三里屯街拍两位好看的闺蜜有人告诉我绑在腿上的带子是啥吗1.jpg
保存成功../crawl_file/images/toutiao/北京三里屯街拍图精选3.jpg
保存成功../crawl_file/images/toutiao/北京三里屯街拍图精选2.jpg
保存成功../crawl_file/images/toutiao/北京三里屯街拍图精选1.j